In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing.pool import ThreadPool
from threading import Lock

In [2]:
data = pd.read_stata("./endkql21mjhqyjk5.dta")

In [3]:
funds = data["fundno"].unique().tolist()

In [4]:
fulldate = []
for year in range(1997,2008):
    for date in ['-03-31', '-06-30', '-09-30', '-12-31']:
        fulldate.append("{0}".format(year)+date)
fulldate = pd.to_datetime(fulldate)

In [15]:
def fund_interp(fundno):
    D = []
    df = data.query("fundno=={0}".format(fundno))
    cusips = df["cusip"].unique().tolist()
    for cusip in cusips:
        dfc = df.query("cusip=='{0}'".format(cusip)).set_index("fdate").reindex(fulldate).interpolate("pad").reset_index()
        D.append(dfc.dropna())
    D = pd.concat(D)
    return D

In [16]:
class Interpolate:
    '''用于输出'''
    def __init__(self,data,result,lock=None):
        self.data = data
        self.result = result
        self.pb = tqdm(total=data["fundno"].unique().shape[0],desc="Interping")
        self.lock = lock

    def __call__(self,fundno):
        res = fund_interp(fundno)
        if self.lock
            self.lock.acquire()
            self.result.append(res)
            self.lock.release()
        else:
            self.result.append(res)
        self.pb.update(1)

In [17]:
D = []
Interpolater = Interpolate(data,D)
pool = ThreadPool(6)
pool.map(Interpolater, funds)
pool.close()

Interping:   0%|          | 0/39971 [00:00<?, ?it/s]

In [18]:
D = pd.concat(D)
D.to_stata("插值.dta")